## 1번째 강의

## solidity 함수

함수는 built-in(제공되는) 함수와 user-defined(사용자가 정의하는) 함수가 있다. 

- 함수의 선언방법

> function (parameter types) {internal | external} [pure | view | payable] [returns (return types)] varName;

1. 생성자 constructor() 
생성자는 컨트랙의 객체를 생성함. 소유자가 누구인지를 설정할 때나, 처음에 설정이 필요한 변수를 초기화할 경우 사용한다. (단, 객체를 만들 때 마다 매번 초기화 되는 게 아니다.) 최초의 한번만 초기화가 된다.

- 생성자는 구현하지 않아도 된다. 그럴 땐 기본 생성자가 주어진다.
- 생성자는 단 1개만 허용된다. overloading(함수이름은 동일하나 인자만 다른 함수)가 불가능하다. 
- 보통 public으로 선언하고, 함수 내에서만 쓰려면 internal로 정의해준다.
> contructor() public{}

2. view와 pure함수
상태를 변경하지 않는 함수는 view나 pure로 표기한다. 이는 local에서만 실행되는 것이므로 블록체인의 transactions이 일어나지 않는다. view는 read only 함수이다.

pure는 제약이 많다. 함수 내에서 지역변수만을 쓰고, 수정 불가능하고, 읽지도 않는 경우에 쓴다. 

⇒ view나 pure가 아닌 경우에는 transactoin에 무엇인가를 한다는 것.
이때에는 반환값이 없고, transaction 해시를 반환하게 된다.
만약 반환값을 이용해서 무엇인가를 하고 싶다면 event를 통해 남겨진 값을 로그에서 확인해다 써야한다.

pure가 할 수 없는 것
- 상태변수 state variables를 읽거나
- 컨트랙의 잔고를 확인할 때 this.balance 같은 것
- 전역변수 block, tx, msg를 읽는 경우
- pure로 표기되지 않는 함수 호출 시
- opcodes로 작성된 경우

view를 사용할 수 없는 경우
- view는 읽는 것이다. 때문에 상태 변수에 저장하는 것
- event가 발생하는 경우(로그에 쓰는 것이므로 안됨)
- 다른 컨트랙 생성 및 삭제 시에도 불가
- call함수로 송금 시
- view or pure가 아닌 함수를 호출하는 경우
- 어셈블리 코드(opcodes)로 작성된 코드

블록체인에 무엇인가를 쓰는 함수
- send(), transfer(), callValue()


payable 함수 
: wei든 Ether든 입출금하는 경우에 사용된다. 
web3를 호출하는 측에서 {value:0}에 ether를 입력하면 msg.value로 전달된다. 컨트랙에서 호출하는 경우는 value()를 쓴다. 

금액을 적을 때에는 반드시 payable함수를 이용해야한다. 컨트랙 호출 시에는 call과 value를 이용해서 괄호 안에 금액을 적어준다.


가시성
: public, internal, private, external
public은 외부에서 누구나 사용이 가능하다.
internal은 블록체인 내부에서 쓰이거나 상속관계에서 쓰임
private은 자신만 사용 가능함.
external은 외부에서만 사용할 수 있음.

함수 배치시 순서 유의하기
: 생성자 / fallback / external / public / internal / private

## modifier
함수의 제약조건으로, 어떤 함수가 실행되려면 만족해야 하는 조건을 modifier로 정할 수 있다. 

## 2번째 강의 
BankV3 스마트 컨트랙을 만들고,배포, 사용해보기

### 1단계 : 컨트랙 개발

In [1]:
%%writefile src/BankV3.sol

pragma solidity ^0.5.0;

contract BankV3 {
    address owner;
    uint balance; //contract의 잔고를 추적하기 위해서 쓰임
    uint256 timeToDeposit; //10초 흘러가는지 확인하기 위한 변수
    
    event PrintLog(string); //event
    event Sent(address from, address to, uint amount);
    constructor() public {
        owner = msg.sender; //초기화를 메세지를 전송하는 주소로 설정(전역변수)
        balance = 0; //초기값
    }
    function() external { //fallback,호출 했는데 해당하는 함수가 없을 경우 실행됨
        emit PrintLog("Fallback called");
    }
    function forwardTo(address payable _receiver) public payable onlyOwner { //인자는 받는 사람, 실행은 owner일 경우에만 하도록 modifier 1설정
        _receiver.transfer(msg.value);//receiver로 msg.value만큼 전송
        emit Sent(msg.sender, _receiver, msg.value); //forawrdTo(계좌이체)가 일어났을 땐 Sent라는 event를 발생시킴
        
    }
    function getBalance() public view returns(uint, uint) { //잔고를 읽는 함수 
        return (balance, address(this).balance); //두개의 잔고 반환 수동으로 추적하는 balance와, 자동으로 추적하는 this.balance
    }
    
    /*금액을 넘겨줘서 deposit하는 경우, 원래는 매개변수가 없어도 value라는 field에 금액을 넣으면 deposit이 됨
    매개변수를 비워놔도 되지만, value field와 amount field를 일치시키기 위함*/
    function deposit(uint amount) public payable onlyAfter { //modifier 2
        timeToDeposit = now + 10 seconds; //10초 지난 것을 여기서 측정함
        require(msg.value == amount); //값이 동일한 경우에만 실행됨. 동일하지 않으면 예외 발생
        balance += amount; //deposit되면 balance를 추적해야함
    }
    //전액인출하는 함수
    function withdrawAll() public onlyOwner minBalance { //오직 Owner만, 최소 잔액이 있어야만 전액 인출할 수 있음, modifier 3설정 
        balance -= address(this).balance; //전체 출금하니 이 금액만큼 잔고에서 제거
        msg.sender.transfer(address(this).balance); // 출금
    }
    modifier onlyOwner {
        require(msg.sender == owner); // msg.sender가 owner일때만 한다
        _; // 호출하는 소스코드
    }
    modifier onlyAfter {
        require(now >= timeToDeposit); // now가 10초 더한 timeTOdeposit과 크거나 같아야 함
        _; // 소스코드가 합쳐지고 나서 실행되도록 함
    }
    modifier minBalance { // 잔고가 101wei보다 클 경우에만 실행
        require(address(this).balance > 101 wei); // 소스코드가 여기에 합쳐지도록 함
        _;
    }
    
}

Writing src/BankV3.sol


### 2단계 : 컴파일

In [2]:
!solc --gas --abi --bin src/BankV3.sol


======= src/BankV3.sol:BankV3 =======
Gas estimation:
construction:
   26125 + 201200 = 227325
external:
   fallback:	1790
   deposit(uint256):	41947
   forwardTo(address):	infinite
   getBalance():	1027
   withdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006001819055506103ee806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe0146100b657806327d8ad88146100e8578063853828b61461012c578063b6b55f2514610143575b34801561004b57600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100c257600080fd5b506100cb610171565b604051808381526020018281526020019250505060405180910390f35b61012a600480360360208110156100fe57600080fd5b81019080803573ffffffffffffffffffffffffffff

### 3단계 : 컨트랙 배포

이번에는 async, await로 변경하여 코드를 작성해보자. 배포 함수를 async function deploy()로 선언한다.
우선 deploy()함수를 만들고, 그 안에 계정 구하는 것, await로 계정을 구해서 넣어본다.

전송한 후 금액 확인도 await send()함수로 만들어 잔고가 즉각 반영되도록 해보자.

In [4]:
%%writefile src/BankV3Deploy.js

var Web3 = require('web3'); //라이브러리설정
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
//var _abiArray = JSON.parse(_abiStr); //js의 dictionary
var _abiArray = [{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
//key:value로 이루어진 독특한 notation임
var _bin = "0x" + "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060006001819055506103ee806100686000396000f3fe60806040526004361061003f5760003560e01c806312065fe0146100b657806327d8ad88146100e8578063853828b61461012c578063b6b55f2514610143575b34801561004b57600080fd5b507f968f0302429ff0e5bd56a45ce3ba1f4fa79f4b822857e438616435f00c3b59fd60405180806020018281038252600f8152602001807f46616c6c6261636b2063616c6c6564000000000000000000000000000000000081525060200191505060405180910390a1005b3480156100c257600080fd5b506100cb610171565b604051808381526020018281526020019250505060405180910390f35b61012a600480360360208110156100fe57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610180565b005b34801561013857600080fd5b506101416102c2565b005b61016f6004803603602081101561015957600080fd5b8101908080359060200190929190505050610381565b005b60008060015447915091509091565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146101d957600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f1935050505015801561021f573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a150565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161461031b57600080fd5b6065471161032857600080fd5b476001600082825403925050819055503373ffffffffffffffffffffffffffffffffffffffff166108fc479081150290604051600060405180830381858888f1935050505015801561037e573d6000803e3d6000fd5b50565b60025442101561039057600080fd5b600a42016002819055508034146103a657600080fd5b806001600082825401925050819055505056fea265627a7a723158201106f1b5227d80d31f8d10d35fae0849693ef12f217bf379ebfbd69ae67e68c764736f6c63430005100032";
//binary는 16진수로.
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() { //async로 선언, 그 안에 promise 반환하는 것을 await해서 결과값을 받아내겠다는 것
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]); //deploy하고있는 coinbase출력
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin}) //blockchain하고, blockchain에 send하는
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        }) //send가 되고나면 결과가 deploy에 들어가도록 함
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address) //contract이 어디에 deployed되고있는지 출력
}
deploy()

Overwriting src/BankV3Deploy.js


In [5]:
!node src/BankV3Deploy.js

Deploying the contract from 0x850B458BE3986cf87a9ff6b117eA19287A57B93A
hash: 0x48444383fe03942c4e96be1e33e8806cf78adec2a3357259a8c35fd20f47b79a
---> The contract deployed to: 0x87f92560f0575D55909E719C4a5458Bbf5d0Bcf9


---> The contract deployed to:address는 복사해놓고 사용할 때 써야함.

### 4단계 : 사용
10초 이내 저축, 잔고101보다 적은데 출금 ⇒  설정한 제약조건을 지켜서 실행해야함

async방식으로 짜여짐. 블록체인에 send()가 필요한 함수는 비동기적으로 처리하기 위해 await으로 처리함. 비동기적으로 처리하면, 입금 deposit()하고 getBalance()시 잔고에 입금분만큼 반영이 됨

In [7]:
%%writefile src/BankV3Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
var _abiArray = [{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"","type":"string"}],"name":"PrintLog","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Sent","type":"event"},{"payable":false,"stateMutability":"nonpayable","type":"fallback"},{"constant":false,"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdrawAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abiArray, "0x87f92560f0575D55909E719C4a5458Bbf5d0Bcf9");

//event를 생성할 때 만들어주는 코드
//var filter = bank.PrintLog(function (error, result) {
//  if (!error)
//    console.log(result);
//});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));

async function doIt() { //async로 doIt함수를 만들었음
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    bank.methods.getBalance().call().then(console.log); //버전3의 잔고 출력
    await bank.methods.deposit(111).send({from: accounts[0], value:111}); //send함수가 있으니 await처리
          //deposit의 111은 그냥 아라비아 숫자인 것, value에 적는 111은 내 잔고에서 빠져서 내잔고로 deposit되는 것 즉 컨트랙으로 입금이 되는것
          //await로 프로그래밍했기 때문에, 잔고가 반영되어 있을 것 
    bank.methods.getBalance().call().then(console.log);
    await bank.methods.withdrawAll().send({from: accounts[0]});    //전액인출, greater than 101
    bank.methods.getBalance().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    // '+'가 숫자를 만나면 연산을, 문자열을 만나면 합성을 함
}
doIt()

Overwriting src/BankV3Use.js


In [8]:
!node src/BankV3Use.js

Account: 0x850B458BE3986cf87a9ff6b117eA19287A57B93A
Balance before: 99994137180000000000
Result { '0': '0', '1': '0' }
Result { '0': '111', '1': '111' }
Result { '0': '0', '1': '0' }
Balance after: 99992464860000000000
Balance diff: 1672320000000000


Result에서 앞의 '0'은 Balance고, '1'은 this.Balance임

일치하면 오류가 없는 것.

차액은 wei만큼 발생

## 3번째 강의
## Events와 Logs

어떤 함수가 실행되는 경우 이벤트가 발생되는 것으로 함. 

함수가 호출되고, 함수 내의 emit 명령어로 이벤트가 발생. 그러면 이벤트는 로그에 기록이 되고, 그로부터 이벤트가 인식됨. EVM의 로그 기능을 사용함.

로그는 blockchain에 저장된다. 이벤트는 상속이 가능하기에 부모가 정의한 이벤트를 자식이 물려받아 발생시킬 수 있다. 

- 이벤트 생성방법
  - 1. 이벤트 설정. 함수명은 CamelCase로 적어주고, 인자를 넣을 수 있으며, 인자 3개까지 로그에서 인덱스로 사용가능하다. 인자의 해시 값이 ### 로그의 topic ### 으로 출력됨
    > event PrintLog()
    
  - 2. 함수를 호출할 때 이벤트가 발생하도록 연결. 이 연결을 'binding'바인딩이라고 함
    > function fireEvent() {
         emit PrintLog() 
       .... }
       
  - 3. 이벤트 발생을 callback 함수로 Listening함. instance.event.PrintLog(), 인자가 있어도 생략 가능. JavaScript callbacks를 사용하여 이벤트가 호출되는지 리스닝함. 이는 2가지 방법이 있다.
        - 이벤트를 생성하고 리스닝하기 - 1) 따로쓰는 방법
        
        > var myEvent = myInstance.events.PrintLog({from: web3.eth.accounts[0]}, {
            fromBlock : 0, //어느 block부터 들을것인지를 명세할 수 있음
            toBlock : 'latest'
           });
        
        > myEvent.watch(function (error, result) { //watch도 할 수 있다.
            if (!error) {
                console.log("Event triggered ===> ", result);
                process.exit(1);
            }
          });
        
        - 이벤트를 생성하고 리스닝하기 - 2) 하나로 합쳐서 쓰는 방법
        
        > var myEvent = myInstance.events.PrintLog({fromBlock : 0}, 
          fromfunction(error, event) { //이벤트 명과 옵션을 열어서 block을 줌)
              if(!error) //그리고 callback 함수
                  console.log(event);
          });
        
  - 4. 함수 호출로 이벤트 발생. 이벤트를 가지고 있는 함수 호출, _instance.MyFunction() or _instance.MyEvent.watch()
    > myInstance.fireEvent();
    
  - 5. 이벤트 watch중지
    > myEvent.stopWatching();

### 간단한 이벤트 만들기 node, listening 이용하기

### 1단계 : 컨트랙 개발

In [15]:
%%writefile  src/EventTest.sol
pragma solidity ^0.6.0;
contract EventTest {
    event MyLog(string my);
    function myFunction() public {//function
        emit MyLog("Hello World!"); //event는 emit, 이벤트 명은 MyLog()
    } // 결과는 deploy후 로그창에서 디버그를 누르고, log부분을 확인해본다.
}

Overwriting src/EventTest.sol


### 2단계 : 컴파일

In [16]:
!solc --abi --bin --gas src/EventTest.sol


======= src/EventTest.sol:EventTest =======
Gas estimation:
construction:
   93 + 42600 = 42693
external:
   myFunction():	1748
Binary:
6080604052348015600f57600080fd5b5060d58061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea26469706673582212207ebc81d2197f982c8d84cdda84be086849eb3d5beaa6ae96a90b729e9e36205764736f6c63430006030033
Contract JSON ABI
[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}]


### 3단계 : 배포

In [17]:
%%writefile src/EventTestDeploy.js

var Web3 = require('web3'); //라이브러리 Import
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
//var _abiArray = JSON.parse(_abiStr);
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="0x"+"6080604052348015600f57600080fd5b5060d58061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c3780a3a14602d575b600080fd5b60336035565b005b7f5186edd9beca61d795526ca1f274260b3fc74be3e10e1f02e1be1552e14f137360405180806020018281038252600c8152602001807f48656c6c6f20576f726c6421000000000000000000000000000000000000000081525060200191505060405180910390a156fea26469706673582212207ebc81d2197f982c8d84cdda84be086849eb3d5beaa6ae96a90b729e9e36205764736f6c63430006030033";

async function deploy() { //deploy함수 생성
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]); //배열로 coinbase출력
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin}) //deploy하고 blockchain으로 보냄. gas가 사용이 되지 않으면 반환이 됨.
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance) {
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address) //디폴로이된 주소 출력
}
deploy()

Overwriting src/EventTestDeploy.js


 geth(사설망)에서 배포하려면 계정 암호를 풀고, 마이닝을 해주어야 password or unlock 오류가 발생하지 않는다.

In [18]:
!node src/EventTestDeploy.js

Deploying the contract from 0x8fe93287ea92f1Ae4FbF820F4aB68765cAdcC0E4
hash: 0xc4059909ac68e3ca933d923820681e43b6916e6e63153f837ab59033be01dc62
---> The contract deployed to: 0x91778E70F909FAE0F1B86Aa6C479d789518F1A25


위에서 주어진 hash값으로 처리결과를 알 수 있다. gas의 사용량, contractAddress도 찾을 수 있다. 

In [21]:
!geth --exec "eth.getTransactionReceipt('0x15ba0c993eb58531eaf61ed290b3e95ddc0f66388b3cb94ad91acd0281cbb717');" attach http://127.0.0.1:8345

{
  blockHash: "0xd3b4a0c684b9f93090811e42df93489b16f8b477bc3c5f6abb7924557a68c764",
  blockNumber: 1,
  contractAddress: "0xeff184d9d68005d8895c7f1de6e68bf9bb563459",
  cumulativeGasUsed: 99005,
  from: "0xe05147cc72a1cb37cf02c7dfb777184e532f0aa6",
  gasUsed: 99005,
  logs: [],
  logsBloom: "0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
  status: "0x1",
  to: null,
  transactionHash: "0x15ba0c993eb58531eaf61ed290b3e95ddc0f66388b3cb94ad91acd0281cbb717",
  transactionIndex: 0
}


 cumulativeGasUsed: 99005 가 사용된 gas의 비용이다. (바이너리 코드의 값까지도 산정해서 나온 결과값이다.)


### 4단계 : 사용

'''JSON.stringify'''는 Object의 내용을 문자열로 출력함

In [10]:
%%writefile src/EventTestHttpNoEventFiredUse.js
// 이벤트가 출력되지 않는 코드(HttpProvider때문)

var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345")); 
//HttpProvider를 하게 되면 event가 fire되지 않음. 이는 안되는 걸 보여주기 위함
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _test = new web3.eth.Contract(_abiArray, '0xe49D9895997E895D897783Bc4661f027CA6A1199');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    //callback함수는 error first 구문으로 작성해준다. result는 MyLog를 실행하게되면 나오는 JSON결과가 result로 출력됨. 
    if (!error) { //이벤트명, 이는 가나슈이니 fromblock이 0부터임.
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    } //JSON.stringify(result.내용을 출력해서 보게 됨) 뒤에있는 result만 출력해서 봐도 됨. 
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction() 
    //앞서 만든 인스턴스 이름의 methods.함수이름 적고, 이 함수를 await(비동기 방식으로 실행), 결과를 value를 받음.
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called "+JSON.stringify(value)+
        //                               '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called "+JSON.stringify(value)+ //위의 const value를 받음
        '\n---> '+ JSON.stringify(value.events.MyLog.returnValues)); //하나의 field를.. 
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
} //잔고 처리 전 후 빼기 등
doIt()

Overwriting src/EventTestHttpNoEventFiredUse.js


In [11]:
!node src/EventTestHttpNoEventFiredUse.js

Account: 0xCd42b098B9Aa8bbc5A6ACe7a938FD31Ae39bC857
Balance before: 99999900779000000000
---> myFunction called {"transactionHash":"0xff17b6d6b6f98913abb94f9fc48c22344118127623033c570cb42abfa0629b71","transactionIndex":0,"blockHash":"0x11bf4413775ad0764a53ae2d21bf25ddef82226f87b53b7de3791c93ea07a753","blockNumber":2,"from":"0xcd42b098b9aa8bbc5a6ace7a938fd31ae39bc857","to":"0xe49d9895997e895d897783bc4661f027ca6a1199","gasUsed":22812,"cumulativeGasUsed":22812,"contractAddress":null,"status":true,"logsBloom":"0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000002000100000000000000000000000000000000000040000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000000

이벤트가 출력되지 않는다. 때문에 HttpProvider가 아닌 WebsocketProvide로 변경해서 출력해보자.

### _abi에 넣어주는 값은 배포시 나온
---> The contract deployed to: 0xeFf184d9D68005D8895C7F1dE6e68bf9bb563459 값을 넣어준다.

In [1]:
%%writefile src/EventTestWsUse.js
//websocket으로 변경 후 출력이 되는 코드(week12 원본)

var Web3=require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _test = new web3.eth.Contract(_abiArray, '0xeFf184d9D68005D8895C7F1dE6e68bf9bb563459');
var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
    if (!error) {
        console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called "+JSON.stringify(value)+
        //                               '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called "+JSON.stringify(value)+
        '\n---> '+ JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}
doIt()

Overwriting src/EventTestWsUse.js


## 파일에 로그쓰기

In [19]:
%%writefile src/EventTestWsUse.js
//week13 업데이트 내용

var Web3=require('web3');
var fs=require('fs');
var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"my","type":"string"}],"name":"MyLog","type":"event"},{"inputs":[],"name":"myFunction","outputs":[],"stateMutability":"nonpayable","type":"function"}];

async function doIt() {
    var _test = new web3.eth.Contract(_abiArray, '0x91778E70F909FAE0F1B86Aa6C479d789518F1A25');
    var event = _test.events.MyLog({fromBlock: 0}, function (error, result) {
        if (!error) {
            log = JSON.stringify(result.returnValues);
            console.log("Event fired: " + log);
            //fs.writeFile("src/EventTestLog.txt", log, "utf-8", function(e)
                           {
            fs.appendFile("src/EventTestLog.txt", log, "utf-8", function(e) {
                if(!e) {
                    console.log(">> Writing to file");
                }
            });
        }
    });
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.myFunction()
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));});
    console.log("---> myFunction called " + JSON.stringify(value.events.MyLog.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1);        
}

doIt()

Overwriting src/EventTestWsUse.js


In [20]:
!node src/EventTestWsUse.js

Account: 0x8fe93287ea92f1Ae4FbF820F4aB68765cAdcC0E4
Balance before: 99999900779000000000
Event fired: {"0":"Hello World!","my":"Hello World!"}
>> Writing to file
---> myFunction called {"0":"Hello World!","my":"Hello World!"}
Balance after: 99999877967000000000
Balance diff: 22812000010240


In [21]:
!type C:\Users\15Z970-G.AA5BK\Code\jsr\src\EventTestLog.txt

{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}{"0":"Hello World!","my":"Hello World!"}


이벤트를 발생할때에는 꼭 Websocket을 이용해야 한다.
결과에서 Event가 Fired되고 있고,
returnValues만하면 ---> {"0":"Hello World!","my":"Hello World!"}가 나옴을 확인가능



### week12 lab의 사용 원본 코드이다. 값을 하나만 넣을때에는 이 코드를 사용하면 됨

In [ ]:
%%writefile src/Multiply7EventUse.js 
//원본
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    var web3 = new Web3(new Web3.providers.WebsocketProvider("http://127.0.0.1:8345"));
}
var _abiArray=[{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"param1","type":"address"},{"indexed":false,"internalType":"uint256","name":"param2","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"param3","type":"uint256"}],"name":"Print","type":"event"},{"constant":false,"inputs":[{"internalType":"uint256","name":"param4","type":"uint256"}],"name":"multiply","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}];
//var _abiArray=JSON.parse(_abiStr);
var _test = new web3.eth.Contract(_abiArray, '0x1D201B7f94B32B458de40249946ae3D86bf245FB');
var event = _test.events.Print({fromBlock: 0}, function (error, result) {
    if (!error) {
        //console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
        console.log("Event fired: " + JSON.stringify(result.returnValues));
        //process.exit(1);
    }
});

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    const value = await _test.methods.multiply(8)
        .send({from: accounts[0], gas: 364124, gasPrice: '1000000000'})
        //.then(function(value) {console.log("---> MyFunction called " + JSON.stringify(value) +
        //                               '\n---> '+ JSON.stringify(value.events.Print.returnValues));});
    console.log("---> MyFunction called " + JSON.stringify(value) +
        '\n\n---> '+ JSON.stringify(value.events.Print.returnValues));
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()